In [1]:


import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np



In [2]:
df = pd.read_csv('data.csv')
df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [3]:

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
import nltk

df = normalize_text(df)
df.head()



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akkua\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akkua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment
0,every great gangster movie current human drama...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino partner crime roger avary co...,negative
4,sat tv hoping name would worth time dear gussi...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,every great gangster movie current human drama...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino partner crime roger avary co...,0
4,sat tv hoping name would worth time dear gussi...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:


import dagshub

mlflow.set_tracking_uri('https://dagshub.com/AXSHATJHA/Capstone-Project.mlflow')
dagshub.init(repo_owner='AXSHATJHA', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")



❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

C:\Users\akkua\anaconda3\envs\PythonProject-Capstone\lib\site-packages\rich\live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=04ed1b1b-e364-47c6-92c0-857993e0b36f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=014eadd6cb592e02542348600a15b009685db6bf9172ffc211554d0ccefe0aaf




Accessing as AXSHATJHA

Initialized MLflow to track repo "AXSHATJHA/Capstone-Project"

Repository AXSHATJHA/Capstone-Project initialized!

2025/10/17 21:23:17 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7352667251c9490692b2f1f6ae1d3feb', creation_time=1760716398005, experiment_id='0', last_update_time=1760716398005, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-10-17 22:55:32,872 - INFO - Starting MLflow run...
2025-10-17 22:55:35,498 - INFO - Logging preprocessing parameters...
2025-10-17 22:55:36,653 - INFO - Initializing Logistic Regression model...
2025-10-17 22:55:36,655 - INFO - Fitting the model...
2025-10-17 22:55:36,699 - INFO - Model training complete.
2025-10-17 22:55:36,699 - INFO - Logging model parameters...
2025-10-17 22:55:37,114 - INFO - Making predictions...
2025-10-17 22:55:37,121 - INFO - Calculating evaluation metrics...
2025-10-17 22:55:37,148 - INFO - Logging evaluation metrics...
2025-10-17 22:55:38,707 - INFO - Saving and logging the model...
2025/10/17 22:55:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-10-17 22:55:39,625 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\akkua\AppData\Local\Temp\ipykernel_18136\2979654972.py", line 48, in 

🏃 View run monumental-mule-498 at: https://dagshub.com/AXSHATJHA/Capstone-Project.mlflow/#/experiments/0/runs/5c2ce88799e74f80a8edaa76a0fafd0f
🧪 View experiment at: https://dagshub.com/AXSHATJHA/Capstone-Project.mlflow/#/experiments/0
